# Imports

In [ ]:
import os
import sys
import pandas as pd
import datetime
import subprocess
import os
import json


In [ ]:
src_dir = os.path.join(os.getcwd(), '..', 'src')
sys.path.append(src_dir)

In [ ]:
from traffic_analysis.d00_utils.data_retrieval import delete_and_recreate_dir
from traffic_analysis.d00_utils.load_confs import load_parameters, load_paths
from traffic_analysis.d02_ref.load_video_names_from_s3 import load_video_names_from_s3
from traffic_analysis.d02_ref.retrieve_and_upload_video_names_to_s3 import retrieve_and_upload_video_names_to_s3
from traffic_analysis.d02_ref.upload_annotation_names_to_s3 import upload_annotation_names_to_s3
from traffic_analysis.d03_processing.update_s3_processed import update_frame_level_table

params = load_parameters()
paths = load_paths()

# Analysis

In [ ]:
#retrieve_and_upload_video_names_to_s3(ouput_file_name='test_search',
#                                      paths=paths,
#                                      from_date='2019-06-30',
#                                      to_date='2019-06-30',
#                                      from_time='13-00-00',
#                                      to_time='13-05-00',
#                                      camera_list=params['tims_camera_list'][:2])

In [ ]:
#upload_annotation_names_to_s3(paths)

In [ ]:
selected_videos = load_video_names_from_s3(ref_file='test_search',
                                           paths=paths)

In [ ]:
chunk_size = params['chunk_size']

In [ ]:
chunk_size

In [ ]:
selected_videos

In [ ]:
update_frame_level_table(file_names=selected_videos[6:10],
                         paths=paths,
                         params=params)

In [ ]:
from traffic_analysis.d03_processing.update_s3_processed import load_s3_parquet

In [ ]:
df = load_s3_parquet(file='frame_table', paths=paths)
df.head()

In [ ]:
df.loc[(df.camera_id == '00001.04336') & (df.frame_id == 0)]

In [ ]:
len(df.camera_id.unique())

In [ ]:
from sqlalchemy import create_engine
db_host = parameters['DATABASE_PARAMS']['db_host']
db_name = parameters['DATABASE_PARAMS']['db_name']
db_user = credentials['dev_sql']['username']
db_pass = credentials['dev_sql']['password']

conn = create_engine('postgresql://%s:%s@%s/%s' % 
                       (db_user, db_pass, db_host, db_name),
                       encoding = 'latin1', 
                       echo = True)

In [ ]:
df.to_sql(name='frame_level',con=conn)

In [ ]:
query = """
select *
from cases
limit 10;
"""

df = pd.read_sql(query, conn)
df.head()

In [ ]:
import os
import yaml

with open(src_dir + '/../conf/local/credentials.yml') as f:
    credentials = yaml.safe_load(f)


In [ ]:
print("{}/{}".format(_bucket_name, load_key))

In [ ]:
from s3fs.core import S3FileSystem
_credentials = credentials
_s3 = S3FileSystem(client_kwargs=_credentials['dev_s3'])
_bucket_name = 's3://' + paths['bucket_name']
load_key = 'frame_level/frame_table/'

with _s3.open("{}/{}".format(_bucket_name, load_key), mode="rb") as s3_file:
    df = pd.read_parquet(s3_file)